In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

Define parameters

In [ ]:
# Which scribbles to use
mode = "all"
bins = [0.01] #, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = 2 #1
# all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
# suff = all_suff[:1]
s = "a"

# Which prediction models to use
convpaint_layers = [[0]]#, [0,2]]
convpaint_scalings = [[1,2]]#[[1], [1,2], [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True

# Where to find and save the data
folder_path = "./cellpose_train_imgs"

Loop and analyse

In [ ]:
df_list = []

for img_num in [1]: #range(0, 540, 10):
    for bin in bins:
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in convpaint_layers:
            for scalings in convpaint_scalings:
                print(f"ConvPaint: layers {layers}, scalings {scalings}")
                pred_tag = generate_convpaint_tag(layers, scalings)
                res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred_tag, show_res=False)
                df_list.append(res)
        if use_ilastik:
            print("Ilastik")
            res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="ilastik", show_res=False)
            df_list.append(res)
        if use_dino:
            print("DINO")
            res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag="dino", show_res=False)
            df_list.append(res)

df = pd.concat(df_list, ignore_index=True)
df[:5]

In [ ]:
df.groupby(['prediction type'])[['accuracy']].mean()
df

Save the data frame in a csv file:

In [ ]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"cellpose_{time_stamp}.csv"
df.to_csv(file_name, index=False)